# 📘 Projet Python : Facturation automatique dans Notion via API
## Par Imane Benamar
---
Ce notebook présente pas à pas la construction d'un système de facturation automatique basé sur une base de données Notion.

Les étapes couvrent la récupération des données, l'analyse, la génération de facture et la mise à jour dans Notion.

In [7]:
# ✅ Étape 1 – Chargement des clés d'accès
from dotenv import load_dotenv
import os

load_dotenv()
NOTION_TOKEN = os.getenv('NOTION_TOKEN')
DB_INTERVENTIONS_ID = os.getenv('DB_INTERVENTIONS_ID')
DB_INVOICES_ID = os.getenv('DB_INVOICES_ID')
print("🔐 Clés chargées avec succès")

🔐 Clés chargées avec succès


In [8]:
# ✅ Étape 2 – Préparer les entêtes Notion API
HEADERS = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
}
print("📡 Headers définis.")

📡 Headers définis.


In [9]:
# ✅ Étape 3 – Requête pour récupérer les interventions non facturées
import requests

def query_unbilled_entries(date_begin: str, date_end: str, a_ete_facture: bool = False):
    query = {
        "filter": {
            "and": [
                {"property": "Facturé", "checkbox": {"equals": a_ete_facture}},
                {"property": "Date de début", "date": {"on_or_after": date_begin}},
                {"property": "Date de début", "date": {"before": date_end}}
            ]
        }
    }

    url = f"https://api.notion.com/v1/databases/{DB_INTERVENTIONS_ID}/query"
    response = requests.post(url, headers=HEADERS, json=query)
    response.raise_for_status()
    return response.json()["results"]

In [10]:
# ✅ Étape 4 – Analyse des données avec pandas
import pandas as pd

def extract_data_from_notion(interventions):
    data = []
    for item in interventions:
        props = item["properties"]
        data.append({
            "Cours": props["Cours"]["title"][0]["text"]["content"] if props["Cours"]["title"] else None,
            "École": props["Ecole"]["select"]["name"] if props["Ecole"]["select"] else None,
            "Classe": props["Classe"]["select"]["name"] if props["Classe"]["select"] else None,
            "Heures": props["Nombre heures"]["number"],
            "Tarif horaire": props["Tarif horaire"]["number"],
            "Total": round(props["Nombre heures"]["number"] * props["Tarif horaire"]["number"], 2)
        })
    return pd.DataFrame(data)

In [11]:
# ✅ Étape 5 – Création d'une facture Notion
def generate_invoice_blocks(interventions, total, client, mois):
    children = [
        {"object": "block", "type": "heading_1", "heading_1": {"rich_text": [{"type": "text", "text": {"content": "FACTURE"}}]}},
        {"object": "block", "type": "paragraph", "paragraph": {"rich_text": [{"type": "text", "text": {"content": f"Client : {client}"}}]}},
        {"object": "block", "type": "paragraph", "paragraph": {"rich_text": [{"type": "text", "text": {"content": f"Mois : {mois}"}}]}},
        {"object": "block", "type": "divider", "divider": {}},
        {"object": "block", "type": "heading_2", "heading_2": {"rich_text": [{"type": "text", "text": {"content": "Détail des interventions"}}]}},
        {"object": "block", "type": "paragraph", "paragraph": {"rich_text": [{"type": "text", "text": {"content": "Cours | Heures | Tarif | Total"}}]}}
    ]
    for item in interventions:
        props = item["properties"]
        cours = props["Cours"]["title"][0]["text"]["content"]
        h = props["Nombre heures"]["number"]
        t = props["Tarif horaire"]["number"]
        ligne = f"{cours} | {h}h | {t}€ | {round(h*t, 2)}€"
        children.append({"object": "block", "type": "paragraph", "paragraph": {"rich_text": [{"type": "text", "text": {"content": ligne}}]}})
    children.append({"object": "block", "type": "callout", "callout": {"icon": {"type": "emoji", "emoji": "💰"}, "rich_text": [{"type": "text", "text": {"content": f"Total à payer : {total} €"}}]}})
    return children

In [12]:
# ✅ Étape 6 – Envoi de la facture dans Notion
def create_invoice_page(client, interventions, total, mois, invoice_number):
    blocks = generate_invoice_blocks(interventions, total, client, mois)
    payload = {
        "parent": {"database_id": DB_INVOICES_ID},
        "properties": {
            "Client": {"title": [{"text": {"content": client}}]},
            "Mois": {"rich_text": [{"text": {"content": mois}}]},
            "Total Amount": {"number": total},
            "Invoice Number": {"rich_text": [{"text": {"content": invoice_number}}]}
        },
        "children": blocks
    }
    res = requests.post("https://api.notion.com/v1/pages", headers=HEADERS, json=payload)
    res.raise_for_status()
    print(f"✅ Facture '{invoice_number}' créée avec succès")

In [13]:
# ✅ Étape 7 – Marquer les interventions comme facturées
def mark_as_billed(pages):
    for page in pages:
        page_id = page["id"]
        update_payload = {"properties": {"Facturé": {"checkbox": True}}}
        response = requests.patch(f"https://api.notion.com/v1/pages/{page_id}", headers=HEADERS, json=update_payload)
        response.raise_for_status()
        print(f"✅ Intervention {page_id} mise à jour.")